In [1]:
# Messing with poisson/wave equation
# from CFD julia using modeling toolkit
using ModelingToolkit
using MethodOfLines
using Plots

In [12]:
# check here: https://docs.sciml.ai/MethodOfLines/stable/tutorials/heat/
# also... 
@parameters x
@variables t u(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)

#2D PDE
C=1
eq  = Dtt(u(t,x)) ~ C^2*Dxx(u(t,x))

# Initial and boundary conditions
bcs = [u(t,0) ~ 0.,# for all t > 0
       u(t,1) ~ 0.,# for all t > 0
       u(0,x) ~ x*(1. - x), #for all 0 < x < 1
       Dt(u(0,x)) ~ 0. ] #for all  0 < x < 1]

# Space and time domains
domains = [t ∈ (0.0,1.0),
           x ∈ (0.0,1.0)]

@named pde_system = PDESystem(eq,bcs,domains,[t,x],[u])

PDESystem
Equations: Equation[Differential(t)(Differential(t)(u(t, x))) ~ Differential(x)(Differential(x)(u(t, x)))]
Boundary Conditions: Equation[u(t, 0) ~ 0.0, u(t, 1) ~ 0.0, u(0, x) ~ (1.0 - x)*x, Differential(t)(u(0, x)) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(t, 0.0 .. 1.0), Symbolics.VarDomainPairing(x, 0.0 .. 1.0)]
Dependent Variables: Symbolics.CallWithMetadata{SymbolicUtils.FnType{Tuple, Real}, Base.ImmutableDict{DataType, Any}}[u⋆]
Independent Variables: Num[t, x]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [10]:
wave = discretize(pde_system, MOLFiniteDifference)

LoadError: MethodError: no method matching discretize(::PDESystem, ::Type{MOLFiniteDifference})

[0mClosest candidates are:
[0m  discretize(::PDESystem, [91m::PDEBase.AbstractEquationSystemDiscretization[39m; analytic, kwargs...)
[0m[90m   @[39m [35mPDEBase[39m [90m~/.julia/packages/PDEBase/jX5Yp/src/[39m[90m[4mdiscretization_state.jl:55[24m[39m


In [11]:
?MOLFiniteDifference

search: MOLFiniteDifference



```
MOLFiniteDifference(dxs, time=nothing;
                    approx_order = 2, advection_scheme = UpwindScheme(),
                    grid_align = CenterAlignedGrid(), kwargs...)
```

A discretization algorithm.

## Arguments

  * `dxs`: A vector of pairs of parameters to the grid step in this dimension, i.e. `[x=>0.2, y=>0.1]`.   For a non-uniform rectilinear grid, replace any or all of the step sizes with the grid you'd like to   use with that variable, must be an `AbstractVector` but not a `StepRangeLen`.
  * `time`: Your choice of continuous variable, usually time. If `time = nothing`, then discretization   yields a `NonlinearProblem`. Defaults to `nothing`.

## Keyword Arguments

  * `approx_order`: The order of the derivative approximation.
  * `advection_scheme`: The scheme to be used to discretize advection terms, i.e. first order spatial derivatives and associated coefficients. Defaults to `UpwindScheme()`. WENOScheme() is also available, and is more stable and accurate at the cost of complexity.
  * `grid_align`: The grid alignment types. See [`CenterAlignedGrid()`](@ref) and [`EdgeAlignedGrid()`](@ref).
  * `use_ODAE`: If `true`, the discretization will use the `ODAEproblem` constructor.   Defaults to `false`.
  * `kwargs`: Any other keyword arguments you want to pass to the `ODEProblem`.


In [16]:
@variables x y u(..) 
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# Poisson equation
# -8*π^2*sin(2*π*x)*sin(2*π*y) - 8*π^2*sin(32*π*x)*sin(32*π*y) 
eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ 0

dirichlet_bcs = [
    u(x, 0) ~ 0., # bottom
    u(x, 1) ~ 1., # top
    u(0, y) ~ 0., # left
    u(1, y) ~ 0.] # right

domains = [
    x ∈ (0.0, 1.0),
    y ∈ (0.0, 1.0)]

@named poisson = PDESystem(
    eq, dirichlet_bcs, domains, [x, y], [u])

PDESystem
Equations: Equation[Differential(y)(Differential(y)(u(x, y))) + Differential(x)(Differential(x)(u(x, y))) ~ 0]
Boundary Conditions: Equation[u(x, 0) ~ 0.0, u(x, 1) ~ 1.0, u(0, y) ~ 0.0, u(1, y) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(x, 0.0 .. 1.0), Symbolics.VarDomainPairing(y, 0.0 .. 1.0)]
Dependent Variables: Symbolics.CallWithMetadata{SymbolicUtils.FnType{Tuple, Real}, Base.ImmutableDict{DataType, Any}}[u⋆]
Independent Variables: Num[x, y]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [10]:
discretizer = MOLFiniteDifference([x=>0.01, y=>0.01])
disc_system = discretize(poisson, discretizer)

LoadError: Sym doesn't have a operation or arguments!

In [15]:
?MOLFiniteDifference

search: MOLFiniteDifference



```
MOLFiniteDifference(dxs, time=nothing;
                    approx_order = 2, advection_scheme = UpwindScheme(),
                    grid_align = CenterAlignedGrid(), kwargs...)
```

A discretization algorithm.

## Arguments

  * `dxs`: A vector of pairs of parameters to the grid step in this dimension, i.e. `[x=>0.2, y=>0.1]`.   For a non-uniform rectilinear grid, replace any or all of the step sizes with the grid you'd like to   use with that variable, must be an `AbstractVector` but not a `StepRangeLen`.
  * `time`: Your choice of continuous variable, usually time. If `time = nothing`, then discretization   yields a `NonlinearProblem`. Defaults to `nothing`.

## Keyword Arguments

  * `approx_order`: The order of the derivative approximation.
  * `advection_scheme`: The scheme to be used to discretize advection terms, i.e. first order spatial derivatives and associated coefficients. Defaults to `UpwindScheme()`. WENOScheme() is also available, and is more stable and accurate at the cost of complexity.
  * `grid_align`: The grid alignment types. See [`CenterAlignedGrid()`](@ref) and [`EdgeAlignedGrid()`](@ref).
  * `use_ODAE`: If `true`, the discretization will use the `ODAEproblem` constructor.   Defaults to `false`.
  * `kwargs`: Any other keyword arguments you want to pass to the `ODEProblem`.
